In [1]:
# pip install domolibrary --upgrade

# MONIT_PageAccess

## CONFIG

set up authentication.  note this script will use your permission.  if you do not have "manage all pages", it will only show content you have permission to see

In [8]:
import domojupyter as dj
import domolibrary.client.DomoAuth as dmda
import json

env = json.loads(dj.get_account_property_value('domo_creds', 'credentials'))

auth =dmda.DomoFullAuth(domo_instance = 'domo-community', 
                  domo_username = env['domo_username'], ## replace this with your username
                  domo_password = env['domo_password'] ## replace this with your password
                 )
await auth.get_auth_token()

## validates that the creds you provided authenticate
assert auth.token

In [3]:
import domolibrary.classes.DomoPage as dmp
import domolibrary.classes.DomoUser as dmu
import domolibrary.classes.DomoGroup as dmg

async def process_page(domo_page :dmp.DomoPage, user_ls: [dmu.DomoUser], group_ls: [dmg.DomoGroup], auth ):
    
    ## get parent page info
    parent_page = None
    top_page = None
    
    if domo_page.parent_page_id:
        parent_page = await dmp.DomoPage.get_by_id(page_id = domo_page.parent_page_id, auth = auth)
    
    if domo_page.top_page_id:
        top_page = await dmp.DomoPage.get_by_id(page_id = domo_page.top_page_id, auth = auth)
    
    ## get group membership (users) for each group a page is shared with
    if group_ls:
        await asyncio.gather(*[ group.Membership.get_members()for group in group_ls])
    
    ## structure data for the dataframe
    group_members_ls = [{'user' : user.email_address, 'group_name': group.name} for group in group_ls for user in group.members_ls if user]

    ## granularity is one row per user per group.  reduce this to one row per user and concat the groups
    if group_members_ls:
        df = pd.DataFrame(group_members_ls)
        df = df.groupby(['user'])['group_name'].apply(', '.join).reset_index()
        group_members_ls = df.to_dict('records')

    # prepare response object
    res= {
        'domo_instance': auth.domo_instance,
        'page_id' : domo_page.id,
        'page_title': domo_page.title,
        'parent_page_id': parent_page.id if parent_page else None,
        'parent_page_title': parent_page.title if parent_page else None,
        'top_page_id' : top_page.id if top_page else None,
        'top_page_title' : top_page.title if top_page else None,
    }
    
    # merge user list with groups they are a member of and res object
    return [ { **res,
       'user_id': user.id,
        'user_email' : user.email_address,
              # 'role_id': user.role_id,
       'group_name': next((group.get('group_name') for group in group_members_ls if group.get('user') == user.email_address), None)
      } for user in user_ls] if user_ls else [res]


In [16]:
import httpx

instance_session = httpx.AsyncClient()

domo_pages = await dmp.DomoPages.get_pages(auth = auth, 
                                           debug_api = True, 
                                           session = instance_session)

## gets all pages from your instance that you have access to
domo_pages[0:5]

[DomoPage(id=1316566624, title='20210623_TRAINING_DomoStats Activity Log App', parent_page_id=127044793, top_page_id=522373865, owners=[], cards=[], collections=None, children=[], is_locked=False, layout=None),
 DomoPage(id=384424178, title='75th Percentile Test', parent_page_id=None, top_page_id=None, owners=[], cards=[], collections=None, children=[], is_locked=False, layout=None),
 DomoPage(id=1917664953, title='A', parent_page_id=1603030697, top_page_id=2054898013, owners=[], cards=[], collections=None, children=[], is_locked=False, layout=None),
 DomoPage(id=1113451501, title='Aditya Jain Dev', parent_page_id=None, top_page_id=None, owners=[], cards=[], collections=None, children=[], is_locked=True, layout=None),
 DomoPage(id=1760351050, title='Adoption Example', parent_page_id=1363798332, top_page_id=307648701, owners=[], cards=[], collections=None, children=[], is_locked=False, layout=None)]

In [5]:
import asyncio

## for each page, get the accesslist and expands groups to include user membership
access_list = await asyncio.gather( *[page.get_accesslist(is_expand_users = True) for page in domo_pages])

print(access_list[0]['resource'])
print("\n")
access_list[0]['users'][0:5]

{'id': '1316566624', 'type': 'page'}




[DomoUser(id='1216550715', title=None, department=None, display_name='8:26 - go to sleep', email_address='test4@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='128618865', title=None, department=None, display_name='9:02 - no really go to bed', email_address='test6@domo.com', role_id=2, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None),
 DomoUser(id='1628021317', title=None, department=None, display_name='Aaron Dean', email_address='aaron.dean@rxa.io', role_id=2097317660, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_do

In [6]:
import pandas as pd
# for each "access_object" in "access_list" match to a page from domo_pages (all the pages you have access to) and then structure for dataframe

group_page_membership = await asyncio.gather(* [process_page(domo_page =next((domo_page for domo_page in domo_pages if str(domo_page.id) == access_obj.get('resource').get('id'))),
                                           user_ls = access_obj.get('users'),
                                           group_ls = access_obj.get('groups'),
                                           auth = auth) for access_obj in access_list])


df =pd.DataFrame([user_member for page in group_page_membership for user_member in page])
df[0:5]

,domo_instance,page_id,page_title,parent_page_id,parent_page_title,top_page_id,top_page_title,user_id,user_email,group_name
0,domo-community,1316566624,20210623_TRAINING_DomoStats Activity Log App,127044793,JaeW_AtOnyx,522373865,Learn Domo,1216550715,test4@domo.com,Default
1,domo-community,1316566624,20210623_TRAINING_DomoStats Activity Log App,127044793,JaeW_AtOnyx,522373865,Learn Domo,128618865,test6@domo.com,Default
2,domo-community,1316566624,20210623_TRAINING_DomoStats Activity Log App,127044793,JaeW_AtOnyx,522373865,Learn Domo,1628021317,aaron.dean@rxa.io,Default
3,domo-community,1316566624,20210623_TRAINING_DomoStats Activity Log App,127044793,JaeW_AtOnyx,522373865,Learn Domo,1833256765,aaron.schofield@domo.com,Default
4,domo-community,1316566624,20210623_TRAINING_DomoStats Activity Log App,127044793,JaeW_AtOnyx,522373865,Learn Domo,269129820,asorensen@emocha.com,Default


# Main Function

In [7]:
import domojupyter as domo
domo.write_dataframe(df, 'monit_user_accesslist')